In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# Task 1 Data Preprocessing and Exploratory Data Analysis

We perform the following steps:
1. Load the dataset ("Dataset.csv") and verify its integrity.
2. Confirm that there are no missing values.
3. Identify and analyze outliers using visualizations such as boxplots.
4. Visualize feature distributions with histograms and KDE plots to understand the
overall distribution of each feature.
5. Review feature statistics (e.g., mean, standard deviation) to get insights into the
data.
6. Normalize or standardize the dataset so that all features contribute equally in
distance calculations, which is crucial for clustering.

### Subtask 1: Load the dataset ("Dataset.csv") and verify its integrity.

Manual inspection of the dataset determined that there are 900 rows (excluding the header row) and 8 columns. There to satisfy the integrity requirement we take that to mean the row and column counts are equal after the dataframe is loaded.







In [ ]:
df = pd.read_csv("Dataset.csv") # load the dataset
rows, cols = df.shape # get the row and column counts
print(f"Dataset shape: {rows} rows, {cols} columns") 

# programmatic verification of the integrity of the dataset, throw an error if the row or column counts are not equal to 900 and 8 respectively
if rows != 900:
    assert False, "The number of rows in the dataset is not equal to 900"
if cols != 8:
    assert False, "The number of columns in the dataset is not equal to 8"

print("Dataset integrity verified")

### Subtask 2: Confirm that there are no missing values.
Count the number of missing values in each column and throw an error if any are found.

In [ ]:
missing_values_count = df.isnull().sum()
if missing_values_count.sum() > 0:
    assert False, "The dataset contains missing values!!!! FIX"
print("Good, No missing values")

### Subtask 3: Identify and analyze outliers using visualizations such as boxplots.
Boxplots for each numerical feature to identify and analyze outliers. Calculate and display statistics about potential outliers. This can be done by calculating the IQR and then using that to identify the lower and upper bounds of the outliers.
The label is categorical so not included in outlier detection.


In [ ]:

sns.set_palette('viridis') # set colour scheme

# Get numerical features from the dataset
numerical_features = df.select_dtypes(include=[np.number]).columns

# Create boxplots for each numerical feature
plt.figure(figsize=(16, 10))
for i, feature in enumerate(numerical_features):
    plt.subplot(3, 3, i+1)  # Adjust grid based on number of features
    sns.boxplot(y=df[feature])
    plt.title(f'Boxplot of {feature}')
    plt.tight_layout()

plt.suptitle('Boxplots for Numerical Features to Identify Outliers', fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()

# Calculate and display statistics about outliers
print("Potential outliers analysis:")
for feature in numerical_features:
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)

    # A standard way to detect outliers is to use the IQR (Interquartile Range) then outliers are any values that fall outside of 1.5 times the IQR below Q1 or above Q3
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)][feature]
    print(f"{feature}: {len(outliers)} outliers detected")
    if len(outliers) > 0:
        print(f"  - Min Boundary: {outliers.min():.2f}, Max Boundary {outliers.max():.2f}]")


### Subtask 4: Visualize feature distributions with histograms and KDE plots to understand the overall distribution of each feature.

Seaborn has differing functions for histograms and KDE plots. Use these.

In [ ]:

plt.figure(figsize=(16, 12))


numerical_features = df.select_dtypes(include=[np.number]).columns # list of numerical features

# Display histograms for each numerical feature
for i, feature in enumerate(numerical_features):
    plt.subplot(3, 3, i+1)  # Adjust grid based on number of features
    sns.histplot(df[feature])
    plt.title(f'Histogram of {feature}')
    plt.tight_layout()

plt.suptitle('Feature Distributions with Histograms', fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()

# Create KDE plots for each numerical feature
plt.figure(figsize=(16, 12))

# Get numerical features from the dataset
numerical_features = df.select_dtypes(include=[np.number]).columns

# Create KDE plots for each numerical feature
for i, feature in enumerate(numerical_features):
    plt.subplot(3, 3, i+1)  # Adjust grid based on number of features
    sns.kdeplot(df[feature], fill=True)
    plt.title(f'KDE Plot of {feature}')
    plt.tight_layout()

plt.suptitle('Feature Distributions with KDE Plots', fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()





All features are skewed to either the left or right

### Subtask 5 - Review feature statistics (e.g., mean, standard deviation) to get insights into the data.

In [ ]:
print("Basic Statistics for Numerical Features via Pandas Dataframe describe:")
display(df.describe())

# Calculate additional statistics that aren't in describe()
print("\nAdditional Statistics:")
numerical_stats = pd.DataFrame({
    'Median': df.select_dtypes(include=[np.number]).median(),
    'Skewness': df.select_dtypes(include=[np.number]).skew(),
    'Kurtosis': df.select_dtypes(include=[np.number]).kurt(),
    'IQR': df.select_dtypes(include=[np.number]).quantile(0.75) - df.select_dtypes(include=[np.number]).quantile(0.25),
    'Range': df.select_dtypes(include=[np.number]).max() - df.select_dtypes(include=[np.number]).min()
})
display(numerical_stats)

# Generate a correlation matrix
print("\nCorrelation Matrix:")
correlation_matrix = df.select_dtypes(include=[np.number]).corr()
display(correlation_matrix)

# Plot the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix of Numerical Features')
plt.tight_layout()
plt.show()


It can be seen that lengths and areas are highly correlated, which is expected as area is a function of length.

### Subtask 6 - Normalize or standardize the dataset so that all features contribute equally in distance calculations, which is crucial for clustering.

For every numeric feature, we will normalize it to a range of 0 to 1.

In [ ]:
# Apply Min-Max normalization to scale features to [0,1] range
scaler = MinMaxScaler()

df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

print("Data has been normalized to the range [0,1]")
